In [1]:
import os
os.chdir('..')

In [2]:
import pystache
import html

data_folder = '../data/'
server_folder = './server/'
template_folder = './templates/'

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [10]:
directions = ['grad', 'hess', 'bfgs', 'bfgs-inv', 'hybrid', 'hybrid-inv']
types = ['LS', 'TR', 'LS-ABS', 'TR-ABS']

combinations = {}

# Add all combinations
for d in directions:
    for t in types:
        combinations[t + '_' + d] = {'type': t, 'direction': d}
        
# Remove the unfeasible ones

unfeasible = ['TR_grad', 'TR-ABS_grad', 'LS_hybrid', 'TR_hybrid', 'TR_bfgs-inv', 'TR-ABS_bfgs-inv', 'LS_hybrid-inv', 'TR_hybrid-inv', 'TR-ABS_hybrid-inv']

for unf in unfeasible:
    del combinations[unf]

In [11]:
combinations

{'LS_grad': {'type': 'LS', 'direction': 'grad'},
 'LS-ABS_grad': {'type': 'LS-ABS', 'direction': 'grad'},
 'LS_hess': {'type': 'LS', 'direction': 'hess'},
 'TR_hess': {'type': 'TR', 'direction': 'hess'},
 'LS-ABS_hess': {'type': 'LS-ABS', 'direction': 'hess'},
 'TR-ABS_hess': {'type': 'TR-ABS', 'direction': 'hess'},
 'LS_bfgs': {'type': 'LS', 'direction': 'bfgs'},
 'TR_bfgs': {'type': 'TR', 'direction': 'bfgs'},
 'LS-ABS_bfgs': {'type': 'LS-ABS', 'direction': 'bfgs'},
 'TR-ABS_bfgs': {'type': 'TR-ABS', 'direction': 'bfgs'},
 'LS_bfgs-inv': {'type': 'LS', 'direction': 'bfgs-inv'},
 'LS-ABS_bfgs-inv': {'type': 'LS-ABS', 'direction': 'bfgs-inv'},
 'LS-ABS_hybrid': {'type': 'LS-ABS', 'direction': 'hybrid'},
 'TR-ABS_hybrid': {'type': 'TR-ABS', 'direction': 'hybrid'},
 'LS-ABS_hybrid-inv': {'type': 'LS-ABS', 'direction': 'hybrid-inv'}}

In [12]:
general_params = [
    {
        'model': 'MNL',
        'name': 'MNL',
        'repetitions': 20,
        'batch': 100,
        'scitas': False,
        'time': '24'
    },
    {
        'model': 'Nested',
        'name': 'Nested',
        'repetitions': 20,
        'batch': 100,
        'scitas': False,
        'time': '24'
    },
    {
        'model': 'LPMC_DrivingCost',
        'name': 'LPMC_DrivingCost_small',
        'repetitions': 20,
        'batch': 1000,
        'scitas': True,
        'time': '12',
        'additions': ', file=\'12.csv\''
    },
    {
        'model': 'LPMC_DrivingCost',
        'name': 'LPMC_DrivingCost',
        'repetitions': 20,
        'batch': 1000,
        'scitas': True,
        'time': '12'
    },
    {
        'model': 'LPMC_DrivingCost',
        'name': 'LPMC_DrivingCost_big',
        'repetitions': 20,
        'batch': 1000,
        'scitas': True,
        'time': '12',
        'additions': ', file=\'12_13_14.csv\''
    },
    {
        'model': 'LPMC_RemoveRest',
        'name': 'LPMC_RemoveRest_small',
        'repetitions': 20,
        'batch': 1000,
        'scitas': True,
        'time': '24',
        'additions': ', file=\'12.csv\''
    },
    {
        'model': 'LPMC_RemoveRest',
        'name': 'LPMC_RemoveRest',
        'repetitions': 20,
        'batch': 1000,
        'scitas': True,
        'time': '24'
    },
    {
        'model': 'LPMC_RemoveRest',
        'name': 'LPMC_RemoveRest_big',
        'repetitions': 20,
        'batch': 1000,
        'scitas': True,
        'time': '24',
        'additions': ', file=\'12_13_14.csv\''
    },
    {
        'model': 'LPMC_Full',
        'name': 'LPMC_Full_small',
        'repetitions': 20,
        'batch': 1000,
        'scitas': True,
        'time': '36',
        'additions': ', file=\'12.csv\''
    },
    {
        'model': 'LPMC_Full',
        'name': 'LPMC_Full',
        'repetitions': 20,
        'batch': 1000,
        'scitas': True,
        'time': '36',
    },
    {
        'model': 'LPMC_Full',
        'name': 'LPMC_Full_big',
        'repetitions': 20,
        'batch': 1000,
        'scitas': True,
        'time': '36',
        'additions': ', file=\'12_13_14.csv\''
    },
]

In [15]:
for params in general_params:

    if not os.path.exists(server_folder + params['name']):
        os.makedirs(server_folder + params['name'])
        
    # Delete old files
    old_files = os.listdir(server_folder + params['name'])
    
    for item in old_files:
        if item.endswith(".py") or item.endswith(".run"):
            os.remove(os.path.join(server_folder + params['name'], item))    
    
    # Create the run all file
    infile = open(server_folder + params['name'] + '/run_all.sh', 'w+')    

    prep_line = None
    if params['scitas']:
        prep_line = 'sbatch {}.run\n'
    else:
        prep_line = 'python {}.py\n'

    lines = [prep_line.format(c) for c in combinations]
    lines.append(prep_line.format('biogeme'))

    for l in lines:
        infile.write(l)
    
    # Go through all combinations
    for c in combinations:

        params['type'] = combinations[c]['type']
        params['direction'] = combinations[c]['direction']
        params['filename'] = c
        params['algo_name'] = params['type'] + ' and ' + params['direction']

        with open(template_folder + '/optimization_python.mustache', 'r') as infile:
            tmpl = infile.read()

        tmpl_filed = pystache.render(tmpl, params)
        
        tmpl_filed = html.unescape(tmpl_filed)

        with open(server_folder + params['name'] + '/' + params['filename'] + '.py', 'w') as outfile:
            outfile.write(tmpl_filed)
            
        if params['scitas']:
            with open(template_folder + '/optimization_scrum.mustache', 'r') as infile:
                tmpl = infile.read()

            tmpl_filed = pystache.render(tmpl, params)

            with open(server_folder + params['name'] + '/' + params['filename'] + '.run', 'w') as outfile:
                outfile.write(tmpl_filed)
                
    # Write the file using biogeme optimization
    params['filename'] = 'biogeme_bfgs'
    params['algo_name'] = 'biogeme'
    
    with open(template_folder + '/optimization_biogeme.mustache', 'r') as infile:
        tmpl = infile.read()

    tmpl_filed = pystache.render(tmpl, params)

    tmpl_filed = html.unescape(tmpl_filed)

    with open(server_folder + params['name'] + '/' + params['filename'] + '.py', 'w') as outfile:
        outfile.write(tmpl_filed)

    if params['scitas']:
        with open(template_folder + '/optimization_scrum.mustache', 'r') as infile:
            tmpl = infile.read()

        tmpl_filed = pystache.render(tmpl, params)

        with open(server_folder + params['name'] + '/' + params['filename'] + '.run', 'w') as outfile:
            outfile.write(tmpl_filed)
        